In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import json
from sklearn.model_selection import train_test_split
from google.colab import drive
import os

drive.mount('/content/drive')
data_path = '/content/drive/MyDrive/WLASL_dataset'
video_folder = os.path.join(data_path, 'videos')
json_file = os.path.join(data_path, 'WLASL_v0.3.json')

#load in the dataset
with open(json_file,'r') as file_object:
    wlasl_data = json.load(file_object)

def extract_frames(img):
    video_id, frame_rate, img_size = img
    video_path = os.path.join(video_folder, f'{video_id}.mp4')
    if not os.path.exists(video_path):
        return None
    #use opencv for file extractiong: https://www.geeksforgeeks.org/python-program-extract-frames-using-opencv/
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        return None
    frames = []
    frame_count = 0
    while cap.isOpened():
        ret1, frame = cap.read()
        if not ret1:
            break
        if (frame_count%frame_rate==0):
            frame = cv2.resize(frame, img_size)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # convert to grayscale
            frames.append(frame)
        frame_count+=1
    cap.release()
    return video_id, frames

def create_transparent_layered_image(frames):
    if not frames:
        return None
    layered_image = np.zeros_like(frames[0], dtype=np.float32)
    #go through frames
    for frame in frames:
        layered_image += frame.astype(np.float32) * (1.0/len(frames)) #creating transparency per frame
    layered_image = np.clip(layered_image,0,255).astype(np.uint8)
    return layered_image

def save_images(img, output_to='/content/drive/MyDrive/images'):
    os.makedirs(output_to, exist_ok=True)
    for i, data in enumerate(img):
        gloss = data['gloss']
        motion_image = data['motion_image']
        gloss_folder = os.path.join(output_to, gloss)

        os.makedirs(gloss_folder, exist_ok=True)
        layered_filename = os.path.join(gloss_folder, f'layered_image_{i}.jpg')
        try:
            # saving grayscale image
            cv2.imwrite(layered_filename, motion_image)
            print(f"Saved transparency layered image for gloss: {gloss} at {layered_filename}")
        except Exception as e:
            print(f"save failed")

def process(image_to_save):
    gloss = image_to_save['gloss']
    full_set = []
    for instance in image_to_save['instances']:
        full_set.append((instance['video_id'],2,(224, 224)))
    images_to_save = []
    for img in full_set:
        data = extract_frames(img)
        if data:
            video_id, frames = data
            motion_image = create_transparent_layered_image(frames) #apply pre-process
            images_to_save.append({'gloss': gloss, 'video_id': video_id, 'frames': frames, 'motion_image': motion_image})

    return images_to_save

#to process them
images_to_save = []
for image_to_save in wlasl_data[:1000]:  #number of glosses processed
    images_to_save.extend(process(image_to_save)) #process
save_images(images_to_save) #save the image

In [ ]:
  #referenced: https://pytorch.org/vision/0.9/transforms.html
  augtransform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # Optional, depending on the task
    transforms.RandomRotation(10),      # Optional, depending on the task
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Usually necessary
])
  augtransform2 = transforms.Compose([transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),transforms.RandomAffine(10, translate=(0.1, 0.1)),
        transforms.ColorJitter(brightness=0.2, contrast=0.2),transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])